<a href="https://colab.research.google.com/github/AlexKressner/OR_Workshop/blob/main/Lineare_Optimierung_Raffinerieproblem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Raffinerieproblem

In [1]:
# Notwendigen Programminstallationen
# pip als Paketmanager
!pip install -U -q pip
!pip install -q ortools
# Laden des Programms
from ortools.linear_solver import pywraplp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 21.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 62.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.4/302.4 kB 3.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-metadata 1.13.0 requires protobuf<4,>=3.13, but you have protobuf 4.22.3 which is incompatible.


In [2]:
# Solver mit SCIP als Backend.
# SCIP implementiert Simplex, Branch-and-Bound, etc
solver = pywraplp.Solver.CreateSolver('SCIP')

## Indexmengen

In [3]:
J = ['K1','K2','K3','K4'] # Menge der Kraftstoffe
I = ['R1','R2','R3','Gesamtkapa'] # Menge der Produktionsressourcen

In [4]:
J

['K1', 'K2', 'K3', 'K4']

## Entscheidungsvariablen

In [5]:
infinity = solver.infinity()

In [6]:
# Definition der Entscheidungsvariablen
x={}
for j in J: 
    x[j] = solver.NumVar(0.0, infinity, j)
print('Anzahl Entscheidungsvariablen =', solver.NumVariables())

Anzahl Entscheidungsvariablen = 4


## Parameter

In [7]:
c={} # Deckungsbeiträge
c['K1']=5
c['K2']=8
c['K3']=15
c['K4']=3

In [8]:
a={} # Produktionskoeffizienten, a[ij]
a['R1']={'K1':1,'K2':1, 'K3':2,'K4':1,}
a['R2']={'K1':2,'K2':1, 'K3':3,'K4':0,}
a['R3']={'K1':0,'K2':1, 'K3':2,'K4':1,}
a['Gesamtkapa']={'K1':1,'K2':1, 'K3':1,'K4':1,}

In [9]:
b={} # Kapazitäten Produktionsfaktoren
b['R1']=1000
b['R2']=500
b['R3']=1500
b['Gesamtkapa']=2250

## Zielfunktion

In [10]:
# Maximierung der gesamten Deckungsbeiträge
solver.Maximize(sum(c[j]*x[j] for j in J))

## Nebenbedingungen

In [11]:
for i in I:
    solver.Add(sum(a[i][j]*x[j] for j in J)<=b[i])
print('Anzahl Nebenbedingungen =', solver.NumConstraints())

Anzahl Nebenbedingungen = 4


## Berechnung Lösung

In [12]:
status = solver.Solve()

if status == pywraplp.Solver.OPTIMAL:
    print('LÖSUNG:')
    print('Zielfunktionswert (Deckungsbeitrag) =', solver.Objective().Value())
    for j in J:
        print(f'{j} =', round(x[j].solution_value()))
else:
    print('Problem hat keine Lösung')

LÖSUNG:
Zielfunktionswert (Deckungsbeitrag) = 5500.0
K1 = 0
K2 = 500
K3 = 0
K4 = 500
